In [1]:
from keras import backend as K
from keras.applications import inception_v3

K.set_learning_phase(0)
model = inception_v3.InceptionV3(weights='imagenet', include_top=False)

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.


In [2]:
layer_dict = dict([(layer.name, layer) for layer in model.layers])

layer_contributions = {
    'mixed2': 0.2,
    'mixed3': 3.,
    'mixed4': 2.,
    'mixed5': 1.5,
}

loss = K.variable(0)
for layer_name in layer_contributions:
    activation = layer_dict[layer_name].output
    scaling = K.prod(K.cast(K.shape(activation), 'float32'))
    coeff = layer_contributions[layer_name]
    loss += coeff * K.sum(K.square(activation[:, 2: -2, 2: -2, :])) / scaling

In [3]:
dream = model.input
grads = K.gradients(loss, dream)[0]
grads /= K.maximum(K.mean(K.abs(grads)), 1e-7)

outputs = [loss, grads]
fetch_loss_and_grads = K.function([dream], outputs)


def eval_loss_and_grads(x):
    outs = fetch_loss_and_grads([x])
    loss_value = outs[0]
    grad_values = outs[1]
    return loss_value, grad_values

def gradient_ascent(x, iterations, step, max_loss=None):
    for i in range(iterations):
        loss_value, grad_values = eval_loss_and_grads(x)
        if max_loss is not None and loss_value > max_loss:
            break
        print('...Loss value at', i, ':', loss_value)
        x += step * grad_values
    return x

In [4]:
IMAGE_PATH = "./input_image.jpg"
NUM_OCTAVE = 3
OCTAVE_SCALE = 1.4
NUM_ITERATION = 20
GRADIENT_STEP = 0.01
MAX_LOSS = 10

In [5]:
import scipy
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing import image

def load_image_data(image_path):
    img = image.load_img(IMAGE_PATH)
    image_data = image.img_to_array(img)
    image_data = np.expand_dims(image_data, axis=0)
    image_data = inception_v3.preprocess_input(image_data)
    return image_data

def show_image_data(image_data):
    if len(image_data.shape) > 3:
        image_data = np.squeeze(image_data, axis=0)
    plt.imshow(image_data)
    plt.axis('off')
    
def resize_image_data(image_data, size):
    image_data = np.copy(image_data)
    factors = (1, float(size[0])/image_data.shape[1], float(size[1])/image_data.shape[2], 1)
    return scipy.ndimage.zoom(image_data, factors, order=1)

def deprocess_image_data(image_data):
    image_data = np.copy(image_data)
    if K.image_data_format() == 'channels_first':
        image_data = image_data.reshape((3, image_data.shape[2], image_data.shape[3]))
        image_data = image_data.transpose((1, 2, 0))
    else:
        image_data = image_data.reshape((image_data.shape[1], image_data.shape[2], 3))
    image_data /= 2.
    image_data += 0.5
    image_data *= 255.
    image_data = np.clip(image_data, 0, 255).astype('uint8')
    return image_data

def save_image_data(image_data, output_path):
    img = deprocess_image_data(image_data)
    scipy.misc.imsave(output_path, img)
    
def compute_resize_loss(image_data, from_size, to_size):
    loss_image_data = resize_image_data(image_data, from_size)
    loss_image_data = resize_image_data(loss_image_data, to_size)
    full_image_data = resize_image_data(image_data, to_size)
    return full_image_data - loss_image_data

In [6]:
original_image_data = load_image_data(IMAGE_PATH)

original_shape = np.array(original_image_data.shape[1:3])
print("original_shape = {}".format(original_shape))
successive_shapes = [(original_shape / (OCTAVE_SCALE ** i)).astype(int) for i in reversed(range(NUM_OCTAVE))]
print("successive_shapes = {}".format(successive_shapes))

dream_image_data = np.copy(original_image_data)
shrunked_image_data = resize_image_data(original_image_data, successive_shapes[0])
print("shrunked_image_data.shape = {}".format(shrunked_image_data.shape))

original_shape = [ 801 1200]
successive_shapes = [array([408, 612]), array([572, 857]), array([ 801, 1200])]
shrunked_image_data.shape = (1, 408, 612, 3)


In [7]:
for i, shape in enumerate(successive_shapes):
    dream_image_data = resize_image_data(dream_image_data, shape)
    dream_image_data = gradient_ascent(dream_image_data, NUM_ITERATION, GRADIENT_STEP, MAX_LOSS)
    if i > 0:
        resize_loss = compute_resize_loss(original_image_data, successive_shapes[i-1], shape)
        dream_image_data += resize_loss
    save_image_data(dream_image_data, "dream_{}x{}.png".format(shape[0], shape[1]))

...Loss value at 0 : 1.5979091
...Loss value at 1 : 2.0978403
...Loss value at 2 : 2.817465
...Loss value at 3 : 3.616597
...Loss value at 4 : 4.4243116
...Loss value at 5 : 5.2047286
...Loss value at 6 : 5.9306455
...Loss value at 7 : 6.6556473
...Loss value at 8 : 7.311224
...Loss value at 9 : 7.998535
...Loss value at 10 : 8.639427
...Loss value at 11 : 9.241897
...Loss value at 12 : 9.833142


d:\Dev\Miniconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:39: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


...Loss value at 0 : 3.2120316
...Loss value at 1 : 4.525763
...Loss value at 2 : 5.6131287
...Loss value at 3 : 6.5748224
...Loss value at 4 : 7.482852
...Loss value at 5 : 8.339045
...Loss value at 6 : 9.135899
...Loss value at 7 : 9.883201


d:\Dev\Miniconda3\envs\tf\lib\site-packages\scipy\ndimage\interpolation.py:605: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)


...Loss value at 0 : 3.2564037
...Loss value at 1 : 4.5050454
...Loss value at 2 : 5.5751963
...Loss value at 3 : 6.5531178
...Loss value at 4 : 7.4827113
...Loss value at 5 : 8.357246
...Loss value at 6 : 9.192659
...Loss value at 7 : 9.977142
